## Hybrid Search with Amazon RDS and Amazon Aurora PostgreSQL using pgvector

In [ ]:
# Install all the required prerequiste libraries - approx 3 min to complete
%pip install setuptools==65.5.0
%pip install -U pgvector pandarallel boto3 psycopg numexpr

In [ ]:
# Load data from csv
import pandas as pd

df = pd.read_csv('data/marketing_sample_for_amazon_com-ecommerce_10k_data.csv')

df = df[['Uniq Id','Product Name','Category','About Product','Product Specification','Technical Details','Image']]

df = df.dropna(subset=['About Product'])
df = df.fillna('')
df.rename(columns={'Uniq Id': 'id', 
                   'Product Name': 'product_name',
                   'Category':'category',
                   'About Product':'product_description',
                   'Product Specification':'product_specification',
                   'Technical Details':'product_details',
                   'Image':'image_url'}, inplace=True)

df['all_descriptions'] = df['product_description'] + df['product_specification'] + df['product_details']

print("Total number of records : {}".format(len(df.index)))

display(df.head(2))


In [ ]:
# Generate vector embeddings of the product description using Amazon Titan Embeddings model hosted in Amazon Bedrock service

import boto3
import json

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

def generate_embeddings(query):
    
    payLoad = json.dumps({'inputText': query })
    
    response = bedrock_runtime.invoke_model(
        body=payLoad, 
        modelId='amazon.titan-embed-g1-text-02',
        accept="application/json", 
        contentType="application/json" )
    response_body = json.loads(response.get("body").read())
    return(response_body.get("embedding"))
    
description_embeddings = generate_embeddings(df.iloc[1].get('all_descriptions'))

print ("Number of dimensions : {}".format(len(description_embeddings)))

In [ ]:
# Generate embeddings for all the products descriptions - approx 3 min to complete
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=8)

# Generate Embeddings for all the products 
df['description_embeddings'] = df['all_descriptions'].parallel_apply(generate_embeddings)

df.head()

print("Completed generation of embeddings for all the products descriptions")


In [ ]:
# Store all the product data with description text and embeddings into Amazon Aurora PostgreSQL database using pgvector

import psycopg
from pgvector.psycopg import register_vector
import boto3 
import json 
import numpy as np

client = boto3.client('secretsmanager')

response = client.get_secret_value(SecretId='secret/hybrid-search-aurorapg')
database_secrets = json.loads(response['SecretString'])

dbhost = database_secrets['host']
dbport = database_secrets['port']
dbuser = database_secrets['username']
dbpass = database_secrets['password']

dbconn = psycopg.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, connect_timeout=10, autocommit=True)

# Enable the pgvector extension
dbconn.execute("CREATE EXTENSION IF NOT EXISTS vector;")
register_vector(dbconn)

# Create the products table
dbconn.execute("DROP TABLE IF EXISTS products;")

dbconn.execute("""CREATE TABLE IF NOT EXISTS products(
                   id text primary key, 
                   product_name text, 
                   category text, 
                   product_description text, 
                   product_specification text,
                   product_details text,   
                   image_url text,
                   description_text text,
                   description_embeddings vector(1536));""")

# Insert the product data
for _, x in df.iterrows():
    dbconn.execute("""INSERT INTO products
                  (id, product_name, category, product_description, product_specification, product_details, image_url, description_text, description_embeddings) 
                   VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s);""", 
                   (x.get('id'), x.get('product_name'), x.get('category'), x.get('product_description'), x.get('product_specification'), x.get('product_details'), x.get('image_url'), x.get('all_descriptions'), x.get('description_embeddings')))

# Create GIN index for the product description text field for lexical search
dbconn.execute("""CREATE INDEX idx_products_description_text_tsvector ON products
                  USING gin(to_tsvector('english', description_text));""")

# Create HNSW index for the product description embeddings feild for vector similarity search
dbconn.execute("""CREATE INDEX ON products 
                   USING hnsw (description_embeddings vector_cosine_ops) 
                   WITH  (m = 16, ef_construction = 64);""")

dbconn.execute("VACUUM ANALYZE products;")

dbconn.close()
print ("Data has been successfully loaded into Aurora PostgreSQL tables ")


In [ ]:
#  Evaluate PostgreSQL lexical search results
import numpy
from IPython.display import display, Markdown, Latex, HTML


def lexical_search(search_text):
    dbconn = psycopg.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, connect_timeout=10)
    register_vector(dbconn)
    
    r= dbconn.execute("""SELECT id, image_url, product_name, product_description, product_details
                         FROM products 
                         WHERE to_tsvector('english', description_text) @@ plainto_tsquery('english', %s)
                         ORDER BY ts_rank(to_tsvector('english', description_text), plainto_tsquery('english', %s)) DESC
                         LIMIT 5;""",(search_text, search_text)).fetchall()
    
    img_td = ""
    for x in r:
        url = x[1].split("|")[0]
        img_td = img_td + """<tr><td><img src={} width="1000"></td>""".format(url)
        img_td = img_td + """<td style="text-align: left; vertical-align: top;"> <h3>{}</h3> <p>{}</p></td></tr>""".format(str(x[2]),str(x[4]))
       
    display(HTML("""<table>{}</table>""".format(img_td)))
    dbconn.close()

print("Lexical search results for 'halloween decoration':\n")
lexical_search("halloween decoration")


In [ ]:
# Evaluate PostgreSQL vector search results
def similarity_search(search_text):
    
    embedding = numpy.array(generate_embeddings(search_text))
    dbconn = psycopg.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, connect_timeout=10)
    register_vector(dbconn)
    
    r= dbconn.execute("""SELECT id, image_url, product_name, product_description, product_details
                         FROM products 
                         ORDER BY description_embeddings <=> %s limit 5;""",(embedding,)).fetchall()
   
    img_td = ""
    for x in r:
        url = x[1].split("|")[0]
        img_td = img_td + """<tr><td><img src={} width="1000"></td>""".format(url)
        img_td = img_td + """<td style="text-align: left; vertical-align: top;"> <h3>{}</h3> <p>{}</p></td></tr>""".format(str(x[2]),str(x[4]))
       
    display(HTML("""<table>{}</table>""".format(img_td)))
    dbconn.close()

print("Similarity search results for 'halloween decoration':\n")
similarity_search("halloween decoration")

In [ ]:
# Evaluate PostgreSQL hybrid search results

def hybrid_search(search_text):
    embedding = numpy.array(generate_embeddings(search_text))
    dbconn = psycopg.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, connect_timeout=10)
    register_vector(dbconn)

    # prioritizing semantic results (left join)
    r= dbconn.execute("""WITH semantic_results AS (
                             SELECT id, 
                             image_url, 
                             product_name, 
                             product_description, 
                             product_details,
                             1 - (description_embeddings <=> %s) AS semantic_similarity
                             FROM products
                             ORDER BY description_embeddings <=> %s
                             LIMIT 100
                        ),
                        lexical_results AS (
                             SELECT id, 
                             ts_rank(to_tsvector('english', description_text), plainto_tsquery('english', %s)) AS lexical_rank
                             FROM products
                             WHERE to_tsvector('english', description_text) @@ plainto_tsquery('english', %s)
                        )
                        SELECT sr.id, 
                               sr.image_url, 
                               sr.product_name, 
                               sr.product_description,   
                               sr.product_details,
                               sr.semantic_similarity * 0.7 + COALESCE(lr.lexical_rank, 0) * 0.3 AS hybrid_score
                        FROM semantic_results sr
                        LEFT JOIN lexical_results lr ON sr.id = lr.id
                        ORDER BY hybrid_score DESC
                        LIMIT 5;""",(embedding,embedding,search_text,search_text)).fetchall()

    # include all results from semantic and lexical search (outer join) 
    r_outerjoin = dbconn.execute("""WITH semantic_results AS (
  SELECT id, 
         image_url, 
         product_name, 
         product_description, 
         product_details,
         1 - (description_embeddings <=> %s) AS semantic_similarity
  FROM products
  ORDER BY description_embeddings <=> %s
  LIMIT 1000  -- Increased limit to capture more potential matches
),
lexical_results AS (
  SELECT id, 
         ts_rank(to_tsvector('english', description_text), plainto_tsquery('english', %s)) AS lexical_rank
  FROM products
  WHERE to_tsvector('english', description_text) @@ plainto_tsquery('english', %s)
)
SELECT 
    COALESCE(sr.id, lr.id) AS id,
    sr.image_url, 
    sr.product_name, 
    sr.product_description, 
    sr.product_details,
    COALESCE(sr.semantic_similarity, 0) * 0.6 + COALESCE(lr.lexical_rank, 0) * 0.4 AS hybrid_score
FROM semantic_results sr
FULL OUTER JOIN lexical_results lr ON sr.id = lr.id
ORDER BY hybrid_score DESC
LIMIT 5;""",(embedding,embedding,search_text,search_text)).fetchall()

    
    img_td = ""
    for x in r_outerjoin:
        url = x[1].split("|")[0]
        img_td = img_td + """<tr><td><img src={} width="1000"></td>""".format(url)
        img_td = img_td + """<td style="text-align: left; vertical-align: top;"> <h3>{}</h3> <p>{}</p></td></tr>""".format(str(x[2]),str(x[4]))
       
    display(HTML("""<table>{}</table>""".format(img_td)))
    dbconn.close()

print("Hybrid search results for 'halloween decoration':\n")
hybrid_search("halloween decoration")